In [16]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


This notebook contains code to reproduce the following from Mitra, et al.:

**Main Figures:** 
* Figure 3A: Number of mutations/RR by category in siblings/probands
* Figure 3B: Mutation size by category
* Figure 3D: RR by AF bin
* Figure 4C: RR by AF bin/locus selection
* Figure 4D: RR by AF bin/allele selection

**Supplementary Tables:** 
* Supplementary Table 5: excess by phase
* Supplementary Table 6: coding mutations (auto+X)
* Supplementary Table 7: deleterious mutations
* Supplementary Table 8: expansions (auto+X)

**Statistics:**
* Mean num proband autosomal mutations
* Mean num sibling autosomal mutations
* Excess in probands vs. siblings (ttest rel, RR)
* Excess in probands vs. siblings after adjust for paternal age (ttest rel)
* Excess in probands vs. siblings after adjust for paternal age + remove homopolymers(ttest rel)
* Mutation excess in male/female chrX
* RR by category

In [17]:
# Input mutation results
MUTFILE = "/storage/ileena/ssc-gangstr-denovos/denovos_GW_priors_Jan20/combined/SSC_allphases_011720_denovos_GW_priors_Jan20.final_qc_mutations.tab"
NAIVEMUTFILE = "/storage/ileena/ssc-gangstr-denovos/denovos_naive_Jan20/SSC_allphases_011720_denovos_naive_Jan20.final_qc_mutations.tab"

#MUTFILEX = "/storage/ileena/ssc-gangstr-denovos/denovos_GW_priors_Jan20/SSC_allphases_072820_denovos_model_X.denovos.final_qc_mutations.tab"
MUTFILEX = "/storage/ileena/ssc-gangstr-denovos/denovos_chrX_naive_Aug20/SSC_allphases_081420_chrX_naive_X.denovos.final_mutations.tab"
NAIVEMUTFILEX = "/storage/ileena/ssc-gangstr-denovos/denovos_naive_Jan20/SSC_allphases_072820_denovos_naive_X.denovos.final_qc_mutations.tab"

# Make editable in Illustrator
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# Imports
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.stats
import statsmodels.api as sm

# Color keys
colors = ['gray','red','gold','blue','green','purple','brown','salmon','magenta','cyan','darkblue']

# Other data paths
ANNOTFILE = "/storage/mgymrek/ssc-denovos/annotations/hg38_gangstr_MasterAnnotation-rmdup.bed"
AFREQFILE = "/storage/mgymrek/ssc-denovos/allele-freqs-revision/SSC_merged_allele_counts.tab"
HETFILE = "/storage/mgymrek/ssc-denovos/allele-freqs-revision/SSC_merged_het.tab"
AGEFILE = "/storage/ileena/denovos4/metadata/ssc_4phases_ages.csv"
SFARIFILE = "/storage/mgymrek/ssc-denovos/annotations/SFARI-Gene_genes_08-29-2019release_09-11-2019export.csv"
JACKGENES = "/storage/mgymrek/ssc-denovos/annotations/jack_102genes.txt"
CONSTRAINTFILE="/storage/mgymrek/ssc-denovos/mutea/MUTEA_Constraint_hg38_GangSTR.bed"

# Output path info
OUTDIR = "pdfs/"

In [18]:
# Load expansions
ndata = pd.read_csv(NAIVEMUTFILE, sep="\t")
ndata = ndata[(ndata["posterior"]==-1) & (ndata["mutsize"]>=5)] # moderately big expansion events

# Filter ones that occur too many times
expcounts = ndata.groupby(["chrom","pos"], as_index=False).agg({"child": len})
ndata = pd.merge(ndata, expcounts[expcounts["child"]<=3][["chrom","pos"]], on=["chrom","pos"])

# Load main mutations data
alldata = pd.read_csv(MUTFILE, sep="\t")
usefams = set(alldata["family"])

# Concatenate expansions
ndata = ndata[ndata.family.isin(usefams)]
alldata = pd.concat([alldata, ndata])

# Load chrX
alldata_x = pd.read_csv(MUTFILEX, sep="\t")
alldata_x = alldata_x[alldata_x.family.isin(usefams)]
ndata_x = pd.read_csv(NAIVEMUTFILEX, sep="\t")
ndata_x = ndata_x[(ndata_x["posterior"]==-1) & (ndata_x["mutsize"]>=5)] # moderately big expansion events
expcounts = ndata_x.groupby(["chrom","pos"], as_index=False).agg({"child": len})
ndata_x = pd.merge(ndata_x, expcounts[expcounts["child"]<=3][["chrom","pos"]], on=["chrom","pos"])
ndata_x = ndata_x[ndata_x.family.isin(usefams)]
alldata_x = pd.concat([alldata_x, ndata_x]).drop_duplicates()

# Manually remove problematic families. 
# 14151 proband had 130 mutations
# other families have <20 in both proband and sibling
rmfams = [14151, 12434, 12281, 13673, 13351, 13355, 13143]
alldata = alldata[~alldata.family.isin(rmfams)]
alldata_x = alldata_x[~alldata_x.family.isin(rmfams)]

# Load annotation info
annot = pd.read_csv(ANNOTFILE, sep="\t")

# Set up fields
def GetChrom(x):
    if "chr" in str(x): return int(x[3:])
    else: return int(x)
alldata["chrom"] = alldata["chrom"].apply(GetChrom)
annot = annot[annot["chrom"].apply(lambda x: x not in ["X","chrX","Y","chrY"])]
annot["chrom"] = annot["chrom"].apply(GetChrom)
annot["pos"] = annot["start"]

# Merge
adata = pd.merge(alldata, annot, on=["chrom","pos"])

# Get age info
ages = pd.read_csv(AGEFILE)
ages["family"] = ages["fam_id"]
ages["child"] = ages["sample_id"]
ages["phenotype"] = ages["ssc_id"].apply(lambda x: [1,2]["p1" in x])
ages = ages[~np.isnan(ages["father_age_birth"])]

alldata_x = pd.merge(alldata_x, ages[["child","sex"]], how="left")

In [ ]:
# Get Allele freq info
het = pd.read_csv(HETFILE, sep="\t")
het["chrom"] = het["chrom"].apply(GetChrom)

afreq = pd.read_csv(AFREQFILE, sep="\t")
afreq.columns = ["chrom","pos","newallele","count"]
afreq["chrom"] = afreq["chrom"].apply(GetChrom)
afreq = pd.merge(afreq, het[["chrom","pos","total"]], on=["chrom","pos"])
afreq["freq"] = afreq.apply(lambda x: float(x["count"])/x["total"], 1)

fdata = pd.merge(adata, afreq, on=["chrom","pos","newallele"], how="left")
fdata.ix[np.isnan(fdata["freq"]),"freq"] = 0
fdata.ix[np.isnan(fdata["count"]),"count"] = 0
fdata = fdata.drop("total", 1)
fdata = pd.merge(fdata, het[["chrom","pos","het","total"]], on=["chrom","pos"])

finaldata = pd.merge(fdata, ages[["family","child","sex","father_age_birth_years"]], on=["family","child"], how="left")

In [ ]:
AFF_COLOR = "red"
UNAFF_COLOR = "blue"

def GetCounts(df, pt, families, sex=None):
    famcounts = {}
    counts = df[df["phenotype"]==pt]
    if sex is not None:
        counts = counts[counts["sex"]==sex]
    for i in range(counts.shape[0]):
        fam = counts["family"].values[i]
        count = counts["nummut"].values[i]
        famcounts[fam] = count
    mutcounts = [famcounts.get(family, 0) for family in families]
    return mutcounts

def TestBurden(usedata):
    # Get counts
    families = set(usedata["family"])
    child_counts = usedata.groupby(["child","phenotype","family","father_age_birth_years"], as_index=False).agg({"posterior": len})
    child_counts.columns = ["child","phenotype","family","father_age_birth_years","nummut"]
    aff_counts = GetCounts(child_counts, 2, families)
    unaff_counts = GetCounts(child_counts, 1, families)
    
    # Overall
    print("#### No adjustment ####")
    print("Unaff mean=%s"%np.mean(unaff_counts))
    print("Aff mean=%s"%np.mean(aff_counts))
    print("Mean diff=%s"%(np.mean(aff_counts)-np.mean(unaff_counts)))
    print(scipy.stats.ttest_rel(unaff_counts, aff_counts))
    
    # Adjust paternal age
    print("#### Correct for paternal age ####")
    unaff_counts = child_counts[child_counts["phenotype"]==1]
    X = sm.add_constant(np.array(unaff_counts[["father_age_birth_years"]]))
    Y = np.array(unaff_counts["nummut"])
    model = sm.OLS(Y, X).fit()

    # Predict and get residuals for all
    child_counts["pred_nummut"] = model.predict(sm.add_constant(np.array(child_counts[["father_age_birth_years"]])))
    child_counts["resid_nummut"] = child_counts["nummut"]-child_counts["pred_nummut"]

    # Compute p-val
    print(scipy.stats.ttest_rel(child_counts[child_counts["phenotype"]==1].sort_values("family")["resid_nummut"],
            child_counts[child_counts["phenotype"]==2].sort_values("family")["resid_nummut"]))

def PlotMutationSize(fdata_list, fdata_list_labels, figprefix=None):
    sizes_means_list_unaff = []
    sizes_stderrs_list_unaff = []
    sizes_means_list_aff = []
    sizes_stderrs_list_aff = []
    female_sizes_means_list_unaff = []
    female_sizes_means_list_aff = []
    male_sizes_means_list_unaff = []
    male_sizes_means_list_aff = []

    for i in range(len(fdata_list)):
        usedata = fdata_list[i]
        flabel = fdata_list_labels[i]
        print("###### %s ######"%flabel)
        # Mutation size
        usedata["absmutsize"] = usedata["mutsize"].apply(abs)
        sizes_unaff = usedata[usedata["phenotype"]==1]["absmutsize"]
        sizes_aff = usedata[usedata["phenotype"]==2]["absmutsize"]
        print(scipy.stats.mannwhitneyu(sizes_unaff, sizes_aff,  alternative="less"))
        print("Unaff n=%s mean=%s"%(len(sizes_unaff), np.mean(sizes_unaff)))
        print("Aff n=%s mean=%s"%(len(sizes_aff), np.mean(sizes_aff)))

        sizes_means_list_unaff.append(np.mean(sizes_unaff))
        sizes_stderrs_list_unaff.append(np.sqrt(np.var(sizes_unaff)/len(sizes_unaff)))
        sizes_means_list_aff.append(np.mean(sizes_aff))
        sizes_stderrs_list_aff.append(np.sqrt(np.var(sizes_aff)/len(sizes_aff)))
        
        female_sizes_means_list_unaff.append(np.mean(usedata[(usedata["phenotype"]==1)&(usedata["sex"]=="female")]["absmutsize"]))
        female_sizes_means_list_aff.append(np.mean(usedata[(usedata["phenotype"]==2)&(usedata["sex"]=="female")]["absmutsize"]))
        male_sizes_means_list_unaff.append(np.mean(usedata[(usedata["phenotype"]==1)&(usedata["sex"]=="male")]["absmutsize"]))
        male_sizes_means_list_aff.append(np.mean(usedata[(usedata["phenotype"]==2)&(usedata["sex"]=="male")]["absmutsize"]))
        
    fig = plt.figure()
    fig.set_size_inches((5+len(fdata_list), 5))
    ax = fig.add_subplot(111)
    
    # Plot overall
    W = 0.2
    xvals = range(len(fdata_list))
    ax.bar(xvals, sizes_means_list_unaff, yerr=1.96*np.array(sizes_stderrs_list_unaff), color=UNAFF_COLOR, width=W)
    ax.bar([item+W for item in xvals], sizes_means_list_aff, yerr=1.96*np.array(sizes_stderrs_list_aff), color=AFF_COLOR, width=W)
    
    # Plot by sex
    ptsize = 100
    ax.scatter([item-W for item in xvals], male_sizes_means_list_unaff, marker="s", color=UNAFF_COLOR, s=ptsize, edgecolors="black")
    ax.scatter([item-W for item in xvals], female_sizes_means_list_unaff, marker="o", color=UNAFF_COLOR, s=ptsize, edgecolors="black")
    ax.scatter([item+2*W for item in xvals], male_sizes_means_list_aff, marker="s", color=AFF_COLOR, s=ptsize, edgecolors="black")
    ax.scatter([item+2*W for item in xvals], female_sizes_means_list_aff, marker="o", color=AFF_COLOR, s=ptsize, edgecolors="black")

    ax.set_ylabel("Mutation size", size=15)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom');
    ax.set_xticks([item+W/2 for item in xvals])
    ax.set_xticklabels(fdata_list_labels, rotation=90)
    if figprefix is not None:
        fig.savefig(os.path.join(OUTDIR, figprefix+"-mutsize.pdf"))

def PlotMutationCounts(full_fdata, fdata_list, fdata_list_labels, plot=True, figprefix=None):
    fdata = full_fdata
    
    # Get list of families to use
    families = set(fdata["family"])
    aff_female_families = set(fdata[(fdata["phenotype"]==2) & (fdata["sex"]=="female")]["family"])
    unaff_female_families = set(fdata[(fdata["phenotype"]==1) & (fdata["sex"]=="female")]["family"])
    aff_male_families = set(fdata[(fdata["phenotype"]==2) & (fdata["sex"]=="male")]["family"])
    unaff_male_families = set(fdata[(fdata["phenotype"]==1) & (fdata["sex"]=="male")]["family"])

    W = 0.2
    ptsize = 100

    # each on a different axis
    for i in range(len(fdata_list)):
        usedata = fdata_list[i]
        flabel = fdata_list_labels[i]
        print("### Comparing counts for %s #####"%flabel)
        child_counts = usedata.groupby(["child","phenotype","family","father_age_birth_years"], as_index=False).agg({"posterior": len})
        child_counts.columns = ["child","phenotype","family","father_age_birth_years","nummut"]
        child_counts_by_sex = usedata.groupby(["child","phenotype","family","sex"], as_index=False).agg({"posterior": len})
        child_counts_by_sex.columns = ["child","phenotype","family","sex","nummut"]

        aff_counts = GetCounts(child_counts, 2, families)
        unaff_counts = GetCounts(child_counts, 1, families)
        aff_male_counts = GetCounts(child_counts_by_sex, 2, aff_male_families, sex="male")
        aff_female_counts = GetCounts(child_counts_by_sex, 2, aff_female_families, sex="female")
        unaff_male_counts = GetCounts(child_counts_by_sex, 1, unaff_male_families, sex="male")
        unaff_female_counts = GetCounts(child_counts_by_sex, 1, unaff_female_families, sex="female")

        print(scipy.stats.ttest_rel(unaff_counts, aff_counts))

        if not plot: continue
        fig = plt.figure()
        fig.set_size_inches((4, 6))
        ax = fig.add_subplot(111)
        ax.bar([0], [np.mean(unaff_counts)], yerr=[1.96*np.sqrt(np.var(unaff_counts)/len(families))], color=UNAFF_COLOR, width=W)
        ax.bar([W], [np.mean(aff_counts)], yerr=[1.96*np.sqrt(np.var(aff_counts)/len(families))], color=AFF_COLOR, width=W)

        ax.scatter([-1*W], [np.mean(unaff_male_counts)], marker="s", color=UNAFF_COLOR, s=ptsize, edgecolors="black")
        ax.scatter([-1*W], [np.mean(unaff_female_counts)], marker="o", color=UNAFF_COLOR, s=ptsize, edgecolors="black")
        ax.scatter([2*W], [np.mean(aff_male_counts)], marker="s", color=AFF_COLOR, s=ptsize, edgecolors="black")
        ax.scatter([2*W], [np.mean(aff_female_counts)], marker="o", color=AFF_COLOR, s=ptsize, edgecolors="black")
        ax.set_xticks([W/2])
        ax.set_ylabel("# mutations/child", size=15)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom');
    
        ax.set_title(fdata_list_labels[i])
        if figprefix is not None:
            fig.savefig(os.path.join(OUTDIR, figprefix+"-burden-%s.pdf"%fdata_list_labels[i]))

def TestSexBurdenMW(fdata, usedata, sex):
    aff_families = set(fdata[(fdata["phenotype"]==2) & (fdata["sex"]==sex)]["family"])
    unaff_families = set(fdata[(fdata["phenotype"]==1) & (fdata["sex"]==sex)]["family"])

    child_counts = usedata[usedata["sex"]==sex].groupby(["child","phenotype","family"], as_index=False).agg({"posterior": len})
    child_counts.columns = ["child","phenotype","family","nummut"]
    aff_counts = GetCounts(child_counts, 2, aff_families)
    unaff_counts = GetCounts(child_counts, 1, unaff_families)
    print("num aff children=%s"%len(aff_counts))
    print("num unaff children=%s"%len(unaff_counts))
    print("mean aff=%s"%np.mean(aff_counts))
    print("mean unaff=%s"%np.mean(unaff_counts))
    print(scipy.stats.mannwhitneyu(aff_counts, unaff_counts, alternative="two-sided"))

# Mutation counts/burden case vs. control

In [ ]:
print("Burden - overall")
TestBurden(finaldata)

print("\nBurden - exclude homopolymers")
TestBurden(finaldata[finaldata["period_x"]>1])

print("\nBurden - by phase")
for phase in ["phase1", "phase2", "phase3_1", "phase3_2"]:
    print(phase)
    TestBurden(finaldata[finaldata["phase"]==phase])

print("\nBurden - chrX male")
TestSexBurdenMW(finaldata, alldata_x, sex="male")

print("\nBurden - chrX female")
TestSexBurdenMW(finaldata, alldata_x, sex="female")

# Fig. 3A Mutation count

In [ ]:
# Mutation count by category
fdata_list_annotations = [finaldata]
fdata_list_annotations_nohom = [finaldata[finaldata["period_x"]>1]]
fdata_labels_list_annotations = ["all"]
for col in ['FEATURE-UTR5','FEATURE-UTR3', 'FEATURE-FETALBRAINTSS', 'FEATURE-CODING', 'FEATURE-INTRON', 'FEATURE-FETALBRAINENH']:
    fdata_list_annotations.append(finaldata[(finaldata[col]>0)])
    fdata_list_annotations_nohom.append(finaldata[(finaldata[col]>0) & (finaldata["period_x"]>1)])
    fdata_labels_list_annotations.append(col.split("-")[1])
PlotMutationCounts(finaldata, fdata_list_annotations, fdata_labels_list_annotations, figprefix="Figure3")

# Fig. 3B Relative risk by category

In [ ]:
import random

# binomial method
def GetRRLowHigh(aff_counts, unaff_counts):
    # return [low, high] for 95% CI
    num_unaff = len(unaff_counts)
    num_aff = len(aff_counts)
    total = np.sum(unaff_counts)+np.sum(aff_counts)
    perc = np.sum(aff_counts)/total
    perc_se = np.sqrt(perc*(1-perc)/total)
    perc_low = perc-1.96*perc_se
    perc_high = perc+1.96*perc_se
    if perc_low < 0: perc_low = 0
    if perc_high > 1: perc_high = 1
    try:
        rr_low = num_unaff*1.0/num_aff*perc_low/(1-perc_low)
        rr_high = num_unaff*1.0/num_aff*perc_high/(1-perc_high)
        return rr_low, rr_high
    except: return np.nan, np.nan

"""
def PlotRR(full_fdata, fdata_list, fdata_list_labels, plot=True, maxORy=None, plotsex=False, figsize=None, figprefix="Figure3",
          colors=None):
    fdata = full_fdata
    
    # Get list of families to use
    families = set(fdata["family"])
    aff_female_families = set(fdata[(fdata["phenotype"]==2) & (fdata["sex"]=="female")]["family"])
    unaff_female_families = set(fdata[(fdata["phenotype"]==1) & (fdata["sex"]=="female")]["family"])
    aff_male_families = set(fdata[(fdata["phenotype"]==2) & (fdata["sex"]=="male")]["family"])
    unaff_male_families = set(fdata[(fdata["phenotype"]==1) & (fdata["sex"]=="male")]["family"])

    # Keep track of the percentage of mutations that are cases/ctrls for each category
    RR_all = []
    RR_female = []
    RR_male = []
    
    RR_low_all = []
    RR_low_female = []
    RR_low_male = []
    RR_high_all = []
    RR_high_female = []
    RR_high_male = []
    
    for i in range(len(fdata_list)):
        usedata = fdata_list[i]
        flabel = fdata_list_labels[i]
        
        # Get counts of unaff/aff mutations
        child_counts = usedata.groupby(["child","phenotype","family","father_age_birth_years"], as_index=False).agg({"posterior": len})
        child_counts.columns = ["child","phenotype","family","father_age_birth_years","nummut"]
        child_counts_by_sex = usedata.groupby(["child","phenotype","family","sex"], as_index=False).agg({"posterior": len})
        child_counts_by_sex.columns = ["child","phenotype","family","sex","nummut"]
        aff_counts = GetCounts(child_counts, 2, families)
        unaff_counts = GetCounts(child_counts, 1, families)
        aff_male_counts = GetCounts(child_counts_by_sex, 2, aff_male_families, sex="male")
        aff_female_counts = GetCounts(child_counts_by_sex, 2, aff_female_families, sex="female")
        unaff_male_counts = GetCounts(child_counts_by_sex, 1, unaff_male_families, sex="male")
        unaff_female_counts = GetCounts(child_counts_by_sex, 1, unaff_female_families, sex="female")

        # Get RR for all/male/female
        RR_all.append(np.mean(aff_counts)/np.mean(unaff_counts))
        RR_female.append(np.mean(aff_female_counts)/np.mean(unaff_female_counts))
        RR_male.append(np.mean(aff_male_counts)/np.mean(unaff_male_counts))

        low_all, high_all = GetRRLowHigh(aff_counts, unaff_counts)
        RR_low_all.append(low_all)
        RR_high_all.append(high_all)
        low_male, high_male = GetRRLowHigh(aff_male_counts, unaff_male_counts)
        RR_low_male.append(low_male)
        RR_high_male.append(high_male)
        low_female, high_female = GetRRLowHigh(aff_female_counts, unaff_female_counts)
        RR_low_female.append(low_female)
        RR_high_female.append(high_female)        
        print("#### %s #####"%fdata_list_labels[i])
        print("Num total/male/female=%s/%s/%s"%(np.sum(aff_counts)+np.sum(unaff_counts),
                                                 np.sum(aff_male_counts)+np.sum(unaff_male_counts),
                                               np.sum(aff_female_counts)+np.sum(unaff_female_counts)))

    ########### Print results #########
    for i in range(len(fdata_list)):
        print("##### %s #####"%(fdata_list_labels[i]))
        print("RR - all: %.2f (%.2f-%.2f)"%(RR_all[i], RR_low_all[i], RR_high_all[i]))
        print("RR - male: %.2f (%.2f-%.2f)"%(RR_male[i], RR_low_male[i], RR_high_male[i]))
        print("RR - female: %.2f (%.2f-%.2f)"%(RR_female[i], RR_low_female[i], RR_high_female[i]))
        
    ########### Plot #########
    if not plot: return
    xvals = range(len(fdata_list))
    fig = plt.figure()
    if figsize is None:
        fig.set_size_inches((5+len(fdata_list), 3))
    else:
        fig.set_size_inches(figsize)
    ax = fig.add_subplot(111)
    W2 = 0.2
    RE_err_low = np.array(RR_all)-np.array(RR_low_all)
    RE_err_high = np.array(RR_high_all)-np.array(RR_all)
    ax.errorbar(xvals, RR_all, yerr = [RE_err_low, RE_err_high], color="black", fmt="o")
    if plotsex:
        RE_err_low_male = np.array(RR_male)-np.array(RR_low_male)
        RE_err_high_male = np.array(RR_high_male)-np.array(RR_male)
        RE_err_low_female = np.array(RR_female)-np.array(RR_low_female)
        RE_err_high_female = np.array(RR_high_female)-np.array(RR_female)
        ax.errorbar([item+W2 for item in xvals], RR_male, yerr = [RE_err_low_male, RE_err_high_male], color="green", fmt="o")
        ax.errorbar([item+W2*2 for item in xvals], RR_female, yerr = [RE_err_low_female, RE_err_high_female], color="purple", fmt="o")
    if maxORy is not None:
        ax.set_ylim(bottom=0.5, top=maxORy)
    ax.axhline(y=1, linestyle="solid", linewidth=0.5, color="black")
    if plotsex:
        ax.set_xticks([item+W2 for item in xvals])
    else:
        ax.set_xticks(xvals)
    ax.set_xticklabels(fdata_list_labels, rotation=90)
    ax.set_ylabel("RR(#case/#ctrl)", size=15)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom');
    if figprefix is not None:
        fig.savefig(os.path.join(OUTDIR, figprefix+"-RR.pdf"))
"""

def PlotRR(full_fdata, fdata_list, fdata_list_labels, plot=True, maxORy=None, plotsex=False, figsize=None, figprefix="Figure3",
          colors=None):
    fdata = full_fdata
    
    # Get list of families to use
    families = set(fdata["family"])
    aff_female_families = set(fdata[(fdata["phenotype"]==2) & (fdata["sex"]=="female")]["family"])
    unaff_female_families = set(fdata[(fdata["phenotype"]==1) & (fdata["sex"]=="female")]["family"])
    aff_male_families = set(fdata[(fdata["phenotype"]==2) & (fdata["sex"]=="male")]["family"])
    unaff_male_families = set(fdata[(fdata["phenotype"]==1) & (fdata["sex"]=="male")]["family"])

    # Keep track of the percentage of mutations that are cases/ctrls for each category
    RR_all = []
    RR_female = []
    RR_male = []
    
    RR_low_all = []
    RR_low_female = []
    RR_low_male = []
    RR_high_all = []
    RR_high_female = []
    RR_high_male = []
    
    p_all = []
    p_male = []
    p_female = []
    
    for i in range(len(fdata_list)):
        usedata = fdata_list[i]
        flabel = fdata_list_labels[i]
        
        # Get counts of unaff/aff mutations
        child_counts = usedata.groupby(["child","phenotype","family","father_age_birth_years"], as_index=False).agg({"posterior": len})
        child_counts.columns = ["child","phenotype","family","father_age_birth_years","nummut"]
        child_counts_by_sex = usedata.groupby(["child","phenotype","family","sex"], as_index=False).agg({"posterior": len})
        child_counts_by_sex.columns = ["child","phenotype","family","sex","nummut"]
        aff_counts = GetCounts(child_counts, 2, families)
        unaff_counts = GetCounts(child_counts, 1, families)
        aff_male_counts = GetCounts(child_counts_by_sex, 2, aff_male_families, sex="male")
        aff_female_counts = GetCounts(child_counts_by_sex, 2, aff_female_families, sex="female")
        unaff_male_counts = GetCounts(child_counts_by_sex, 1, unaff_male_families, sex="male")
        unaff_female_counts = GetCounts(child_counts_by_sex, 1, unaff_female_families, sex="female")

        # Get RR for all/male/female
        RR_all.append(np.mean(aff_counts)/np.mean(unaff_counts))
        RR_female.append(np.mean(aff_female_counts)/np.mean(unaff_female_counts))
        RR_male.append(np.mean(aff_male_counts)/np.mean(unaff_male_counts))

        # Get p-val for all/male/female
        p_all.append(scipy.stats.ttest_rel(unaff_counts, aff_counts)[1]/2) # ttest_rel returns 2-sided. convert to 1-sided
        p_male.append(scipy.stats.ttest_ind(unaff_male_counts, aff_male_counts)[1]/2) 
        p_female.append(scipy.stats.ttest_ind(unaff_female_counts, aff_female_counts)[1]/2) 

        low_all, high_all = GetRRLowHigh(aff_counts, unaff_counts)
        RR_low_all.append(low_all)
        RR_high_all.append(high_all)
        low_male, high_male = GetRRLowHigh(aff_male_counts, unaff_male_counts)
        RR_low_male.append(low_male)
        RR_high_male.append(high_male)
        low_female, high_female = GetRRLowHigh(aff_female_counts, unaff_female_counts)
        RR_low_female.append(low_female)
        RR_high_female.append(high_female)        
        print("#### %s #####"%fdata_list_labels[i])
        print("Num total/male/female=%s/%s/%s"%(np.sum(aff_counts)+np.sum(unaff_counts),
                                                 np.sum(aff_male_counts)+np.sum(unaff_male_counts),
                                               np.sum(aff_female_counts)+np.sum(unaff_female_counts)))

    ########### Print results #########
    for i in range(len(fdata_list)):
        print("##### %s #####"%(fdata_list_labels[i]))
        print("RR - all: %.2f (%.2f-%.2f) p=%s"%(RR_all[i], RR_low_all[i], RR_high_all[i], p_all[i]))
        print("RR - male: %.2f (%.2f-%.2f) p=%s"%(RR_male[i], RR_low_male[i], RR_high_male[i], p_male[i]))
        print("RR - female: %.2f (%.2f-%.2f) p=%s"%(RR_female[i], RR_low_female[i], RR_high_female[i], p_female[i]))
        
    ########### Plot #########
    if not plot: return
    xvals = range(len(fdata_list))
    fig = plt.figure()
    if figsize is None:
        fig.set_size_inches((5+len(fdata_list), 3))
    else:
        fig.set_size_inches(figsize)
    ax = fig.add_subplot(111)
    W2 = 0.2
    RE_err_low = np.array(RR_all)-np.array(RR_low_all)
    RE_err_high = np.array(RR_high_all)-np.array(RR_all)
    ax.errorbar(xvals, RR_all, yerr = [RE_err_low, RE_err_high], color="black", fmt="o")
    if plotsex:
        RE_err_low_male = np.array(RR_male)-np.array(RR_low_male)
        RE_err_high_male = np.array(RR_high_male)-np.array(RR_male)
        RE_err_low_female = np.array(RR_female)-np.array(RR_low_female)
        RE_err_high_female = np.array(RR_high_female)-np.array(RR_female)
        ax.errorbar([item+W2 for item in xvals], RR_male, yerr = [RE_err_low_male, RE_err_high_male], color="green", fmt="o")
        ax.errorbar([item+W2*2 for item in xvals], RR_female, yerr = [RE_err_low_female, RE_err_high_female], color="purple", fmt="o")
    if maxORy is not None:
        ax.set_ylim(bottom=0.5, top=maxORy)
    ax.axhline(y=1, linestyle="solid", linewidth=0.5, color="black")
    if plotsex:
        ax.set_xticks([item+W2 for item in xvals])
    else:
        ax.set_xticks(xvals)
    ax.set_xticklabels(fdata_list_labels, rotation=90)
    ax.set_ylabel("RR(#case/#ctrl)", size=15)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom');
    if figprefix is not None:
        fig.savefig(os.path.join(OUTDIR, figprefix+"-RR.pdf"))
        
PlotRR(finaldata, fdata_list_annotations, fdata_labels_list_annotations, maxORy=4, plotsex=True, figprefix="Figure3")
PlotRR(finaldata, fdata_list_annotations_nohom, fdata_labels_list_annotations, maxORy=4, plotsex=True, figprefix="ED-Fig8-NoHomopolymers")


In [ ]:
# Compare if burden different between subdata vs. full data 
def CompareBurden(fulldata, subdata):
    num_total_aff = fulldata[fulldata["phenotype"]==2].shape[0]
    num_total_unaff = fulldata[fulldata["phenotype"]==1].shape[0]
    num_sub_aff = subdata[subdata["phenotype"]==2].shape[0]
    num_sub_unaff = subdata[subdata["phenotype"]==1].shape[0]
    # 2x2 table of mutation in annot/not and in proband/not
    table = [[num_sub_aff, num_total_aff-num_sub_aff],
            [num_sub_unaff, num_total_unaff-num_sub_unaff]]
    return scipy.stats.fisher_exact(table, alternative="greater")

print(CompareBurden(finaldata, finaldata[finaldata["FEATURE-FETALBRAINTSS"]>0]))
print(CompareBurden(finaldata[finaldata["sex"]=="male"], finaldata[(finaldata["sex"]=="male") & (finaldata["FEATURE-FETALBRAINTSS"]>0)]))
print(CompareBurden(finaldata[finaldata["sex"]=="female"], finaldata[(finaldata["sex"]=="female") & (finaldata["FEATURE-FETALBRAINTSS"]>0)]))

# Fig.  3B Mutation size

In [ ]:
# Mutation size by category
PlotMutationSize(fdata_list_annotations, fdata_labels_list_annotations, figprefix="Figure3")

# Figure 4A: Relative risk by AF

In [ ]:
mintotal = 0.8*max(fdata["total"])

# All
fdata_list = [finaldata]
fdata_list_nohom = [finaldata[finaldata["period_x"]>1]]
fdata_labels_list = ["All"]

# Other freq bins
for freqpair in [(0.05, 1.1), (0.01, 0.05), (0.001, 0.01), (-1,0.001), (-1, 0)]:
    minfreq, maxfreq = freqpair
    if minfreq == -1 and maxfreq > 0:
        fdata_list.append(finaldata[(finaldata["total"]>=mintotal) & (finaldata["freq"]>0) & (finaldata["freq"]<=maxfreq)])
        fdata_list_nohom.append(finaldata[(finaldata["period_x"]>1) & (finaldata["total"]>=mintotal) & (finaldata["freq"]>0) & (finaldata["freq"]<=maxfreq)])
        fdata_labels_list.append("Count1-%s"%(maxfreq))
    else:
        fdata_list.append(finaldata[(finaldata["total"]>=mintotal) & (finaldata["freq"]>minfreq) & (finaldata["freq"]<=maxfreq)])
        fdata_list_nohom.append(finaldata[(finaldata["period_x"]>1) &(finaldata["total"]>=mintotal) & (finaldata["freq"]>minfreq) & (finaldata["freq"]<=maxfreq)])
        fdata_labels_list.append("%s-%s"%(minfreq,maxfreq))
        
PlotRR(finaldata, fdata_list, fdata_labels_list, plotsex=False, figprefix="Figure4-AF")
PlotMutationCounts(finaldata, fdata_list, fdata_labels_list, plot=False, figprefix=None)

PlotRR(finaldata, fdata_list_nohom, fdata_labels_list, plotsex=True, figprefix="Figure4-AF-nohom")
PlotMutationCounts(finaldata, fdata_list_nohom, fdata_labels_list, plot=False, figprefix=None)


In [ ]:
print(CompareBurden(finaldata, finaldata[(finaldata["freq"]==0) & (finaldata["total"]>=mintotal)]))
print(CompareBurden(finaldata[finaldata["sex"]=="male"], finaldata[(finaldata["sex"]=="male") & (finaldata["freq"]==0) & (finaldata["total"]>=mintotal)]))
print(CompareBurden(finaldata[finaldata["sex"]=="female"], finaldata[(finaldata["sex"]=="female") & (finaldata["freq"]==0) & (finaldata["total"]>=mintotal)]))

# Figure 4b-c: RR by AF bin/locus selection

In [ ]:
import statsmodels.stats.multitest

mintotal = 0.8*max(fdata["total"])

# Load selection estimates and generate per-allele scores
def GetSelectionScore(x):
    if np.isnan(x["ABC_s_median"]): return np.nan
    diff = abs(x["newallele"]-x["optimal_ru"])
    s_est = x["ABC_s_median"]
    return diff*s_est

#seldata = pd.read_csv("/storage/BonnieH/selection_project/per_locus/results/per_locus_results_prelim_all_per.txt", sep="\t")
SSCFILE = "/storage/BonnieH/selection_project/per_locus/SISTR_results/0.005_3_0.3_yny5_eurodem_prior2_all_per_0810.txt"
seldata = pd.read_csv(SSCFILE, sep="\t")
seldata["pos"] = seldata["start"]
seldata = seldata[~np.isnan(seldata["LRT_p_value"])]
print("Found SISTR data for %s STRs"%seldata.shape[0])
seldata["ABC_upper"] = seldata["ABC_s_95%_CI"].apply(lambda x: float(x.split(",")[1].split(")")[0]))
seldata["ABC_lower"] = seldata["ABC_s_95%_CI"].apply(lambda x: float(x.split(",")[0].split("(")[1]))
seldata["width"] = seldata["ABC_upper"]-seldata["ABC_lower"]
seldata = seldata[seldata.apply(lambda x:  (x["width"]<0.3), 1)] # scores not trustworthy if width is big

print("Found SISTR data for %s STRs after filtering"%seldata.shape[0])

rej, qvals = statsmodels.stats.multitest.fdrcorrection(seldata["LRT_p_value"])
seldata["qvalue"] = qvals

sistr = seldata.copy() # for use downstream, before we merge

# Now merge with mutation data
seldata = pd.merge(finaldata, seldata, on=["chrom","pos"], how="left")
seldata["pscore"] = seldata.apply(GetSelectionScore, 1)
seldata["total"] = seldata["total_x"]

In [ ]:
# All
q_thresh = 0.01

fdata_list = [seldata, seldata[(seldata["qvalue"]<q_thresh) & ~np.isnan(seldata["LRT_p_value"])] ]
fdata_labels_list = ["All", "All-selp"]
colors = ["black","blue"]

# Other freq bins
for freqpair in [(0.05, 1.1), (0.01, 0.05), (0.001, 0.01), (-1,0.001), (-1, 0)]:
    minfreq, maxfreq = freqpair
    if minfreq == -1 and maxfreq > 0:
        fdata_list.append(seldata[(seldata["total"]>=mintotal) & (seldata["freq"]>0) & (seldata["freq"]<=maxfreq)])
        fdata_labels_list.append("Count1-%s"%(maxfreq))
        colors.append("black")
        
        fdata_list.append(seldata[(seldata["qvalue"]<q_thresh) & ~np.isnan(seldata["LRT_p_value"]) &
                                  (seldata["total"]>=mintotal) & (seldata["freq"]>0) & (seldata["freq"]<=maxfreq)])
        fdata_labels_list.append("Count1-%s-selp"%(maxfreq))
        colors.append("blue")

    else:
        fdata_list.append(seldata[(seldata["total"]>=mintotal) & (seldata["freq"]>minfreq) & (seldata["freq"]<=maxfreq)])
        fdata_labels_list.append("%s-%s"%(minfreq,maxfreq))
        colors.append("black")
        
        fdata_list.append(seldata[(seldata["qvalue"]<q_thresh) & 
                (seldata["total"]>=mintotal) & (seldata["freq"]>minfreq) & (seldata["freq"]<=maxfreq)])
        fdata_labels_list.append("%s-%s-selp"%(minfreq,maxfreq))
        colors.append("blue")
        
PlotRR(finaldata, fdata_list, fdata_labels_list, plotsex=False, maxORy=2, figprefix="Figure4-LocusSel", figsize=(6, 4))

# Per allele score, by af
#s_threshs = [0, 10e-5, 10e-4, 5e-3, 0.015]
s_threshs = np.percentile(seldata[~np.isnan(seldata["pscore"])]["pscore"], [0, 90, 99])
for freqbin in [(-0.5,1.1), (0, 0.01), (-1, 0)]:
    print("\n\n%s"%str(freqbin))
    fdata_list = []
    fdata_list_labels = []
    minfreq, maxfreq = freqbin
    for sthresh in s_threshs:
        if sthresh == 0: use_q = 1
        else: use_q = q_thresh
        fdata_list.append(seldata[(seldata["freq"]>minfreq) & (seldata["freq"]<=maxfreq) &
                                  (seldata["qvalue"]<use_q) & 
                                  (seldata["pscore"]>=sthresh) & (seldata["total"]>=mintotal)])
        fdata_list_labels.append("%.6f"%sthresh)
    print(fdata_list_labels)
    PlotRR(finaldata, fdata_list, fdata_list_labels, plotsex=False,
           figprefix="Figure4-AlleleSel-%s"%str(freqbin), figsize=(6, 4))

# Figure 4b - MUTEA

In [ ]:
MUTEAFILE = "/storage/mgymrek/ssc-denovos/mutea/MUTEA_Constraint_hg38_GangSTR.bed"
mutea = pd.read_csv(MUTEAFILE, sep="\t", names=["chrom","pos","end","MUTEA"])
mutea["chrom"] = mutea["chrom"].apply(GetChrom)
mdata = pd.merge(sistr, mutea, on=["chrom","pos"])

mdata = mdata[~np.isnan(mdata["ABC_s_median"])]
plotdata = []

plotdata.append(mdata[mdata["qvalue"]>=0.01]["MUTEA"])
plotdata.append(mdata[mdata["qvalue"]<0.01]["MUTEA"])

fig = plt.figure()
ax = fig.add_subplot(111)
bp = ax.boxplot(plotdata, sym="", positions=range(len(plotdata)), patch_artist=True,
               medianprops={"color": "white"});
for patch in bp["boxes"]:
    patch.set_facecolor("black")
ax.axhline(y=0, color="black", linewidth=0.5)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel("MUTEA Z-score")
ax.set_xticks(range(len(plotdata)))
ax.set_xticklabels(["SISTR q>=0.01", "SISTR q<0.01"], rotation=45);
fig.savefig("pdfs/SISTR_vs_MUTEA.pdf")

import scipy.stats
print(scipy.stats.mannwhitneyu(plotdata[0], plotdata[1], alternative="greater"))
print(len(plotdata[0]))
print(len(plotdata[1]))

# ED Fig. 10d-e

In [ ]:
gnomad = pd.read_csv("/storage/mgymrek/workspace/sistr-analyses/annotations/gnomad.v2.1.1.lof_metrics.by_gene.txt", sep="\t")
gnomad["gene-coding"] = gnomad["gene"]

annot = pd.read_csv("/storage/mgymrek/ssc-denovos/annotations/hg38_gangstr_MasterAnnotation-rmdup.bed", sep="\t")
annot["chrom"] = annot["chrom"].apply(GetChrom)
annot["start"] = annot["pos"]
usedata = pd.merge(sistr, annot, on=["chrom","start"], how="left")

pli = pd.merge(usedata, gnomad[["gene-coding","pLI","oe_lof_upper","mis_z"]], on=["gene-coding"])
pli["color"] = pli.apply(lambda x: ["black","red"][int(x["LRT_p_value"]<0.01)], 1)

In [ ]:
for feature in ["FEATURE-CODING"]:
    pos = pli[(pli["qvalue"]<0.01) & ( (pli[feature]>0)) & ~np.isnan(pli["oe_lof_upper"])]

    neg = pli[(pli["qvalue"]>=0.01) & ((pli[feature]>0)) & ~np.isnan(pli["oe_lof_upper"])]

    print(pos.shape[0])
    print(neg.shape[0])
    al = 0.5
    fig = plt.figure()
    ax = fig.add_subplot(111)
    bins = np.arange(-6, 6, 0.25)
    ax.hist(pos["mis_z"], color="red", alpha=al, bins=bins, normed=True);
    ax.hist(neg["mis_z"], color="black", alpha=al, bins=bins, normed=True);
    ax.axvline(np.median(pos["mis_z"]), color="red")
    ax.axvline(np.median(neg["mis_z"]), color="black")
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    fig.savefig("pdfs/EDFig10_misZ-%s.pdf"%feature)
    print(scipy.stats.mannwhitneyu(pos["mis_z"], neg["mis_z"], alternative="greater"))

    fig = plt.figure()
    ax = fig.add_subplot(111)
    bins = np.arange(0, 1.11, 0.1)
    ax.hist(pos["pLI"], color="red", alpha=al, bins=bins, normed=True);
    #ax.hist(med["pLI"], color="blue", alpha=al, bins=bins, normed=True);
    ax.hist(neg["pLI"], color="black", alpha=al, bins=bins, normed=True);
    ax.axvline(np.median(pos["pLI"]), color="red")
    #ax.axvline(np.median(med["pLI"]), color="blue")
    ax.axvline(np.median(neg["pLI"]), color="black")
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    fig.savefig("pdfs/EDFig10_pli-%s.pdf"%feature)
    print(scipy.stats.mannwhitneyu(pos["pLI"], neg["pLI"], alternative="greater"))


# Supplementary Table 7 - TRs under selection

In [ ]:
cols = ["phenotype","chrom","pos","mutsize","motif_x","pscore","TSS_GENE"]
seldata[(seldata["pscore"]>s_threshs[-1]) & (seldata["qvalue"]<q_thresh) & (seldata["freq"]<=0)].sort_values(["phenotype","pscore"], ascending=False)[cols]

# Supplementary Table 6: Coding mutations

In [ ]:
finaldata[finaldata["FEATURE-CODING"]>0] # autosome only. add chrX separately. should be 32 autosomal coding

# Supplementary Table 8 - expansions

In [ ]:
ndata # add chrX separately.